In [183]:
from bs4 import BeautifulSoup
import sqlite3
import imp
import requests
import pandas as pd
import tqdm
tw = imp.load_source('tw', '../twitter-past-crawler/src/twitterpastcrawler/crawler.py')

## GET RETWEETS AND FAVORITES
https://twitter.com/i/activity/retweeted_popup?id=1025962880280211456

https://twitter.com/i/activity/favorited_popup?id=1025962880280211456

In [84]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10.5; en-US; rv:1.9.0.1) Gecko/2008070206'}
url = 'https://twitter.com/i/search/timeline?l=&q=from:mblivre%20since:2018-08-02%20until:2018-08-03&vertical=default&max_position=hoge&src=typd&include_entities=1&include_available_features=1'

html = requests.get(url=url, headers=headers).json()["items_html"]

soup = BeautifulSoup(html, "lxml")
all_tweets = soup.find_all("li", attrs={"class": "stream-item"})

In [86]:
len(all_tweets)

20

In [94]:
for t in all_tweets:
    if 'is-square' in str(t):
        print(all_tweets.index(t))

0
2
4
16


In [92]:
type(t)

bs4.element.Tag

In [95]:
element = all_tweets[0]

In [96]:
tweet_container = list(element.children)[1]
contents = None
for c in tweet_container.findChildren():
    if tw.has_class(c, "content"):
        contents = c
        break

### Embbed Media

In [113]:
for c in contents.findChildren():
    if tw.has_class(c, "AdaptiveMedia is-video"):
        embbed_type='video'
    if tw.has_class(c, "AdaptiveMedia-photoContainer"):
        embbed_type='image'
        embbed_url = c.attrs['data-image-url']

https://pbs.twimg.com/media/DjohoSnX0AAjU5g.jpg


### Quote

In [50]:
for c in contents.findChildren():
    if tw.has_class(c, "QuoteTweet-text"):
        a = c.text

'Pessoal, o filho de um político ameaçou de morte um comediante e uma mulher. Ele fez um boletim de ocorrência contra o "jornal teleguiado" por causa deste post, que denuncia isso. Por favor, não fiquem compartilhando o post, ok?\n\nhttp://teleguiado.com/humor/2018/08/filho-de-ex-vereador-de-conselheiro-lafaiete-ameaca-de-morte-leo-lins-e-namorada.html\xa0…'

### Hashtag

In [59]:
hashtags = []
for c in contents.findChildren():
    if tw.has_class(c, "twitter-hashtag"):
        hashtags.append(c.text)
hashtags

['#fernandinho',
 '#beiramar',
 '#marcola',
 '#pcc',
 '#zona',
 '#guerra',
 '#crimes',
 '#quadrilhas',
 '#criminosos',
 '#violência',
 '#corrupto',
 '#corrupção',
 '#lavajato',
 '#moro',
 '#lula',
 '#pt']

## Retweets

In [217]:
def get_interaction_username(tweet_id, type_int, db, cursor):
    
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10.5; en-US; rv:1.9.0.1) Gecko/2008070206'}
    if type_int == 'retweet':
        url = 'https://twitter.com/i/activity/retweeted_popup?id={}'.format(tweet_id)
    elif type_int == 'favorited':
        url = 'https://twitter.com/i/activity/favorited_popup?id={}'.format(tweet_id)

    # 'htmlTitle', 'htmlContent', 'htmlUsers'
    try:
        html = requests.get(url=url, headers=headers).json()
    except:
        return

    soup = BeautifulSoup(html['htmlUsers'], "lxml")

    all_users = soup.find_all("span", attrs={"class": "username"})

    users = []
    for a in all_users:
        if a.text not in users:
            users.append(a.text)
            add_data_interaction(tweet_id, type_int, a.text, db, cursor)


def init_table_interaction(db, cursor, force_reset=False):
    
    if force_reset:
        cursor.execute('DROP TABLE IF EXISTS interaction;')

    query = """CREATE TABLE interaction(
                            tweet_id TEXT,
                            interaction_type TEXT,
                            username TEXT
                            )"""
    try:
        cursor.execute(query)
        db.commit()
        print('success')
    except:
        pass

def add_data_interaction(tweet_id, int_type, username, db, cursor):
    
    cursor.execute("""INSERT INTO interaction(
                tweet_id,
                interaction_type,
                username) 
                VALUES(?,?,?)""", 
                (str(tweet_id),
                str(int_type),
                str(username)))
    
    db.commit()
    
def main():
    
    db = sqlite3.connect('../scripts/tweets')
    cursor = db.cursor()
    
    init_table_interaction(db, cursor)
    
    tweets = pd.read_sql_query('SELECT * FROM tweets_mbl', db)
    
    inte = pd.read_sql_query('SELECT * FROM interaction', db)
    tweets = list(tweets[~(tweets['tweet_id'].isin(inte.drop_duplicates(subset=['tweet_id'])['tweet_id']))]['tweet_id'])
    
    print(len(tweets))
    
    for tweet_id in tqdm.tqdm(tweets):
        
        get_interaction_username(tweet_id, 'retweet', db, cursor)
        
        get_interaction_username(tweet_id, 'favorited', db, cursor)

        #cursor.execute('SELECT COUNT(*) FROM interaction')
        #print(cursor.fetchone())

In [218]:
main()

  0%|          | 0/5694 [00:00<?, ?it/s]

5694


100%|██████████| 5694/5694 [2:27:15<00:00,  1.55s/it]  


In [221]:
tweets = pd.read_sql_query('SELECT * FROM interaction', db)

In [223]:
len(pd.read_sql_query('SELECT * FROM tweets_mbl', db)) - len(tweets.drop_duplicates(subset=['tweet_id']))

116